In [1]:
import cv2
import sys
import time
import numpy as np

sys.path.append('../tools')

from fps_utils import run_model

In [2]:
video = '../samples/mot16.webm' 

## YOLACT FPS

In [3]:
# dummy image for initial loading
run_model(np.random.randint(255, size=(900,800,3),dtype=np.uint8))

-> Loading model from  models/yolact_plus_resnet50_54_800000.pth
   Loading pretrained model


In [4]:
cap = cv2.VideoCapture(video)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [5]:
start = time.time()

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    run_model(frame)
    
duration = time.time() - start
    
cap.release()

In [6]:
yolact_fps = n_frames / duration

print(f"frame Size: {h}x{w}\nFPS: {yolact_fps}")

frame Size: 540x960
FPS: 6.069989430966422


## Tracker FPS

In [7]:
from tracktor_utils import tracker_obj

In [8]:
tracker = tracker_obj("../tracking_wo_bnw")
tracker.reset()

In [9]:
cap = cv2.VideoCapture(video)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [10]:
start = time.time()

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    run_model(frame, tracker)
    
duration = time.time() - start
    
cap.release()

In [11]:
tracker_fps = n_frames / duration

print(f"frame Size: {h}x{w}\nFPS: {tracker_fps}")

frame Size: 540x960
FPS: 6.082505448182656
